# Azure AI Content Understanding

This notebook demonstrates how to use the Azure AI Content Understanding service
in Foundry Tools. Content Understanding provides powerful capabilities for analyzing
and extracting information from documents and images.

## Prerequisites

Set the following environment variables:
- `FOUNDRY_CONTENT_UNDERSTANDING_ENDPOINT` - Azure AI Content Understanding service endpoint
- `FOUNDRY_API_KEY` - API key for authentication
- `FOUNDRY_REGION` - Region in which Foundry resources are provisioned

In [ ]:
import json
import os
import time

import requests
from rich.console import Console
from rich.panel import Panel
from rich.table import Table

## API Setup

Configure the Azure AI Content Understanding API endpoint and authentication.

In [ ]:
endpoint = os.environ["FOUNDRY_CONTENT_UNDERSTANDING_ENDPOINT"]
api_key = os.environ["FOUNDRY_API_KEY"]
region = os.environ["FOUNDRY_REGION"]

headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/json",
}

console = Console()

print(f"API Endpoint: {endpoint}")
print(f"Region: {region}")
print("API Key: [CONFIGURED]")

# Capability 1: List Content Analyzers

The Content Understanding service provides various prebuilt analyzers for different
document and image analysis tasks. Let's discover what analyzers are available.

In [ ]:
# List all available content analyzers
list_url = f"{endpoint}/contentunderstanding/analyzers?api-version=2025-11-01"

response = requests.get(list_url, headers=headers, timeout=30)
response.raise_for_status()
analyzers_result = response.json()

print("API call successful!")

## Raw API Results - Available Analyzers

In [ ]:
print(json.dumps(analyzers_result, indent=2))

## Human-Friendly Results - Available Analyzers

In [ ]:
console = Console()

# Create a table of available analyzers
table = Table(title="Available Content Analyzers")
table.add_column("Analyzer ID", style="cyan", no_wrap=True)
table.add_column("Description", style="white")

for analyzer in analyzers_result.get("value", []):
    analyzer_id = analyzer.get("id", "N/A")
    description = analyzer.get("description", "No description available")
    table.add_row(analyzer_id, description)

console.print(table)
console.print(f"\n[bold green]Total Analyzers Available: {len(analyzers_result.get('value', []))}[/bold green]")

# Capability 2: Content Extraction

Extract annotations and highlights from a PDF document using the prebuilt-layout analyzer.
This is useful for extracting marked content, comments, and highlighted sections from documents.

## Prepare PDF Analysis Request

We'll analyze a PDF ebook that contains highlights using the prebuilt-layout analyzer.

In [ ]:
# PDF URL with highlights
pdf_url = "https://www.dropbox.com/scl/fi/405z56tds3tewycemxp23/Azure-AI-Fundamentals-AI-900-Study-Guide-1.pdf?rlkey=i0t4xnrrt392lba4s4fqlt1j7&st=yomn8xte&dl=1"

# Download the PDF file
print(f"Downloading PDF from: {pdf_url}")
pdf_response = requests.get(pdf_url, timeout=60)
pdf_response.raise_for_status()
pdf_data = pdf_response.content
print(f"Downloaded {len(pdf_data)} bytes")

# Endpoint for binary content analysis
analyze_url = f"{endpoint}/contentunderstanding/analyzers/prebuilt-layout:analyzeBinary?api-version=2025-11-01&features=annotations"

# Prepare headers for binary upload
pdf_headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/pdf",
}

print("\nAnalyzer: prebuilt-layout")
print("Features: annotations")

## Make PDF Analysis API Call

Submit the document for analysis. This is an asynchronous operation.

In [ ]:
# Submit the analysis request with binary data
response = requests.post(analyze_url, headers=pdf_headers, data=pdf_data, timeout=30)
response.raise_for_status()

# Get the operation location to poll for results
operation_location = response.headers.get("Operation-Location")
print("Analysis submitted successfully!")
print(f"Operation Location: {operation_location}")

# Poll for results
max_attempts = 30
attempt = 0

while attempt < max_attempts:
    attempt += 1
    time.sleep(2)
    
    result_response = requests.get(operation_location, headers=headers, timeout=30)
    result_response.raise_for_status()
    pdf_result = result_response.json()
    
    status = pdf_result.get("status")
    print(f"Attempt {attempt}: Status = {status}")
    
    if status == "succeeded":
        print("\n✓ Analysis completed successfully!")
        break
    elif status == "failed":
        print("\n✗ Analysis failed!")
        break
    elif attempt >= max_attempts:
        print("\n⚠ Timeout waiting for results")
        break

## Raw API Results - PDF Annotations

In [ ]:
print(json.dumps(pdf_result, indent=2))

## Human-Friendly Results - PDF Annotations

Extract and display the highlighted content and annotations from the PDF.

In [ ]:
console = Console()

if pdf_result.get("status") == "succeeded":
    analyze_result = pdf_result.get("analyzeResult", {})
    
    # Extract annotations
    annotations = []
    for page in analyze_result.get("pages", []):
        page_num = page.get("pageNumber", "Unknown")
        for annotation in page.get("annotations", []):
            annotations.append({
                "page": page_num,
                "kind": annotation.get("kind", "Unknown"),
                "content": annotation.get("content", "")
            })
    
    # Display annotations in a table
    if annotations:
        table = Table(title="PDF Annotations & Highlights")
        table.add_column("Page", style="cyan", justify="center")
        table.add_column("Type", style="yellow")
        table.add_column("Content", style="white")
        
        for ann in annotations:
            table.add_row(
                str(ann["page"]),
                ann["kind"],
                ann["content"][:100] + "..." if len(ann["content"]) > 100 else ann["content"]
            )
        
        console.print(table)
        console.print(f"\n[bold green]Total Annotations Found: {len(annotations)}[/bold green]")
    else:
        console.print("[yellow]No annotations found in the document[/yellow]")
    
    # Display document statistics
    stats_table = Table(title="Document Statistics")
    stats_table.add_column("Metric", style="cyan")
    stats_table.add_column("Value", style="green")
    
    stats_table.add_row("Total Pages", str(len(analyze_result.get("pages", []))))
    stats_table.add_row("Total Paragraphs", str(len(analyze_result.get("paragraphs", []))))
    stats_table.add_row("Total Annotations", str(len(annotations)))
    
    console.print("\n")
    console.print(stats_table)
else:
    console.print(f"[red]Analysis status: {pdf_result.get('status', 'Unknown')}[/red]")

# Capability 3: Classification and Segmentation

Analyze and classify items in images using the prebuilt-image analyzer.
This capability can detect objects, segment regions, and classify content in images.

## Analyze Image 1: Produce

Analyze an image of produce to classify and segment items.

In [ ]:
# Image of produce
produce_image_url = "https://images.unsplash.com/photo-1635341083777-5f93a755e916?q=80&w=500"

# Download the image
image_response = requests.get(produce_image_url, timeout=30)
image_response.raise_for_status()
produce_image_data = image_response.content

print(f"Downloaded produce image: {len(produce_image_data)} bytes")

In [ ]:
# Analyze the produce image
image_analyze_url = f"{endpoint}/contentunderstanding/analyzers/prebuilt-image:analyzeBinary?api-version=2025-11-01"

image_headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/octet-stream",
}

response = requests.post(image_analyze_url, headers=image_headers, data=produce_image_data, timeout=30)
response.raise_for_status()

# Get the operation location
operation_location = response.headers.get("Operation-Location")
print("Image analysis submitted successfully!")
print(f"Operation Location: {operation_location}")

# Poll for results
max_attempts = 30
attempt = 0

while attempt < max_attempts:
    attempt += 1
    time.sleep(2)
    
    result_response = requests.get(operation_location, headers=headers, timeout=30)
    result_response.raise_for_status()
    produce_result = result_response.json()
    
    status = produce_result.get("status")
    print(f"Attempt {attempt}: Status = {status}")
    
    if status == "succeeded":
        print("\n✓ Analysis completed successfully!")
        break
    elif status == "failed":
        print("\n✗ Analysis failed!")
        break
    elif attempt >= max_attempts:
        print("\n⚠ Timeout waiting for results")
        break

## Raw API Results - Produce Image

In [ ]:
print(json.dumps(produce_result, indent=2))

## Human-Friendly Results - Produce Image

In [ ]:
console = Console()

if produce_result.get("status") == "succeeded":
    analyze_result = produce_result.get("analyzeResult", {})
    
    console.print(Panel("[bold cyan]Produce Image Analysis[/bold cyan]", expand=False))
    
    # Extract detected objects/segments
    figures = analyze_result.get("figures", [])
    content_items = analyze_result.get("content", [])
    
    # Display figures/objects detected
    if figures:
        table = Table(title="Detected Objects/Segments")
        table.add_column("ID", style="cyan")
        table.add_column("Caption", style="white")
        table.add_column("Bounding Region", style="yellow")
        
        for figure in figures:
            fig_id = figure.get("id", "N/A")
            caption = figure.get("caption", {}).get("content", "No caption")
            regions = figure.get("boundingRegions", [])
            region_str = f"{len(regions)} region(s)" if regions else "None"
            
            table.add_row(fig_id, caption, region_str)
        
        console.print(table)
    
    # Display content/text detected
    if content_items:
        console.print("\n[bold green]Text Content Detected:[/bold green]")
        for item in content_items[:10]:  # Show first 10 items
            console.print(f"  • {item}")
    
    # Display summary statistics
    stats_table = Table(title="Image Analysis Summary")
    stats_table.add_column("Metric", style="cyan")
    stats_table.add_column("Value", style="green")
    
    stats_table.add_row("Total Figures/Objects", str(len(figures)))
    stats_table.add_row("Total Content Items", str(len(content_items)))
    stats_table.add_row("Pages Analyzed", str(len(analyze_result.get("pages", []))))
    
    console.print("\n")
    console.print(stats_table)
else:
    console.print(f"[red]Analysis status: {produce_result.get('status', 'Unknown')}[/red]")

## Analyze Image 2: Elephants

Analyze an image of elephants to classify and segment items.

In [ ]:
# Image of elephants
elephants_image_url = "https://images.unsplash.com/photo-1505148230895-d9a785a555fa?q=80&w=500"

# Download the image
image_response = requests.get(elephants_image_url, timeout=30)
image_response.raise_for_status()
elephants_image_data = image_response.content

print(f"Downloaded elephants image: {len(elephants_image_data)} bytes")

In [ ]:
# Analyze the elephants image
response = requests.post(image_analyze_url, headers=image_headers, data=elephants_image_data, timeout=30)
response.raise_for_status()

# Get the operation location
operation_location = response.headers.get("Operation-Location")
print("Image analysis submitted successfully!")
print(f"Operation Location: {operation_location}")

# Poll for results
max_attempts = 30
attempt = 0

while attempt < max_attempts:
    attempt += 1
    time.sleep(2)
    
    result_response = requests.get(operation_location, headers=headers, timeout=30)
    result_response.raise_for_status()
    elephants_result = result_response.json()
    
    status = elephants_result.get("status")
    print(f"Attempt {attempt}: Status = {status}")
    
    if status == "succeeded":
        print("\n✓ Analysis completed successfully!")
        break
    elif status == "failed":
        print("\n✗ Analysis failed!")
        break
    elif attempt >= max_attempts:
        print("\n⚠ Timeout waiting for results")
        break

## Raw API Results - Elephants Image

In [ ]:
print(json.dumps(elephants_result, indent=2))

## Human-Friendly Results - Elephants Image

In [ ]:
console = Console()

if elephants_result.get("status") == "succeeded":
    analyze_result = elephants_result.get("analyzeResult", {})
    
    console.print(Panel("[bold cyan]Elephants Image Analysis[/bold cyan]", expand=False))
    
    # Extract detected objects/segments
    figures = analyze_result.get("figures", [])
    content_items = analyze_result.get("content", [])
    
    # Display figures/objects detected
    if figures:
        table = Table(title="Detected Objects/Segments")
        table.add_column("ID", style="cyan")
        table.add_column("Caption", style="white")
        table.add_column("Bounding Region", style="yellow")
        
        for figure in figures:
            fig_id = figure.get("id", "N/A")
            caption = figure.get("caption", {}).get("content", "No caption")
            regions = figure.get("boundingRegions", [])
            region_str = f"{len(regions)} region(s)" if regions else "None"
            
            table.add_row(fig_id, caption, region_str)
        
        console.print(table)
    
    # Display content/text detected
    if content_items:
        console.print("\n[bold green]Text Content Detected:[/bold green]")
        for item in content_items[:10]:  # Show first 10 items
            console.print(f"  • {item}")
    
    # Display summary statistics
    stats_table = Table(title="Image Analysis Summary")
    stats_table.add_column("Metric", style="cyan")
    stats_table.add_column("Value", style="green")
    
    stats_table.add_row("Total Figures/Objects", str(len(figures)))
    stats_table.add_row("Total Content Items", str(len(content_items)))
    stats_table.add_row("Pages Analyzed", str(len(analyze_result.get("pages", []))))
    
    console.print("\n")
    console.print(stats_table)
else:
    console.print(f"[red]Analysis status: {elephants_result.get('status', 'Unknown')}[/red]")

# Summary

This notebook demonstrated three key capabilities of Azure AI Content Understanding:

1. **Content Analyzers Discovery** - Listed all available prebuilt analyzers
2. **Content Extraction** - Extracted annotations and highlights from a PDF document
3. **Image Classification & Segmentation** - Analyzed and classified objects in images

These capabilities can be used for various document and image analysis tasks including:
- Document layout analysis
- Annotation extraction
- Object detection and classification
- Image segmentation
- Content understanding and categorization